In [39]:
#  시스템콜 이름 -> 시스템콜 번호 딕셔너리 만들기
import subprocess
sysname_convert = dict()
sysnum_convert = dict()
with open("/tmp/syscall_64.txt","r") as f:
    for line in f.readlines():
        sysnum = line.strip().split()[0] # sysnum
        if len(line.strip().split()) == 4:
            sysname = line.strip().split()[3] # sysname
            if "compat_sys_" in sysname:
                sysname = sysname.replace("compat_sys_","")
            elif "sys_" in sysname:
                sysname = sysname.replace("sys_","")
            if "stub_x32_" in sysname:
                sysname = sysname.replace("stub_x32_","")
            if "stub_" in sysname:
                sysname = sysname.replace("stub_","")
        else:
            sysname = line.strip().split()[2] # sysname
        # x86 기준, x86 시스템콜에 없는 번호는 x32 기준
        if not sysname in sysname_convert.keys():
            sysname_convert[sysname] = sysnum
            sysnum_convert[sysnum] = sysname

In [89]:
# 기존에 없는 시스템콜 처리
sysname_convert["socketcall"] = "437"
sysnum_convert["437"] = "socketcall"
sysname_convert["vm86old"] = "436"
sysnum_convert["436"] = "vm86old"

# Benign Image

In [406]:
import subprocess

# 모든 benign syscall sequence load
seqList = dict()
imgList = {"httpd", "mongodb", "mysql", "nginx", "redis", "mariadb", "node", "tomcat", "gcc", "openjdk", "gzip", "bzip2", "ghostscript", "lowriter", "qalc"}

for imgName in imgList:
    seqs = subprocess.check_output(f"find ../data/parsing/{imgName}/*.txt", shell=True).decode().strip().split()
    for seq in seqs:
        case = seq.replace(f"../data/parsing/{imgName}/","").replace(".txt","")
        with open(seq, "r") as f:
            syslist = f.readlines()
            syslist = list(map(lambda x: x.strip(), syslist))
            seqList[f"{imgName}_{case}"]=syslist
        
print(len(seqList))

46


In [407]:
# 시스템콜 번호으로 바꾸기
seqNumList = dict()

for case, seq in seqList.items():
    # 변환
    num_seq = list()
    for sys_name in seq:
        num_seq.append(sysname_convert[sys_name])
    seqNumList[case]=num_seq

## System Call Set 분석

In [408]:
# System Call Set 분석
b_sysSet = dict()
for imgName in imgList:
    b_sysSet[imgName] = set()

for case, seq in seqNumList.items():
    imgName = case.split("_")[0]
    for syscall in seq:
        b_sysSet[imgName].add(syscall)
        
b_sysSet_total = set()
for imgName in imgList:
    print(f"image {imgName} syscall set #: {len(b_sysSet[imgName])}")
    b_sysSet_total = b_sysSet_total | b_sysSet[imgName]
print(f"total syscall set #: {len(b_sysSet_total)}")

image tomcat syscall set #: 25
image bzip2 syscall set #: 17
image mysql syscall set #: 39
image gcc syscall set #: 38
image httpd syscall set #: 21
image lowriter syscall set #: 76
image node syscall set #: 13
image nginx syscall set #: 14
image ghostscript syscall set #: 38
image redis syscall set #: 14
image mongodb syscall set #: 45
image gzip syscall set #: 20
image mariadb syscall set #: 22
image openjdk syscall set #: 57
image qalc syscall set #: 42
total syscall set #: 107


In [53]:
# N-gram 생성 함수
def tokenize(trg, N=1):
    a = trg.split(",")                 
    d = []
    for i in range(len(a) - N+1):
        b = a[i:i+N]
        c = (',').join(b)
        d.append(c)
    return d

## Image N-gram 생성

In [409]:
b_ngramList = dict()
for imgName in imgList:
    b_ngramList[imgName] = set()
MAX_N = 65
for case, seq in seqNumList.items():
    imgName = case.split("_")[0]
    seq_str = ",".join(seq)
    for i in range(2, 1+MAX_N):
        ngram_result = tokenize(seq_str, i)
        if len(ngram_result) == 0:
            break
        for ngram in ngram_result:
            b_ngramList[imgName].add(ngram)
        ngram_result = list()
    print(f"image {imgName} ngram #: {len(b_ngramList[imgName])}")

image tomcat ngram #: 12886
image tomcat ngram #: 26740
image tomcat ngram #: 41167
image bzip2 ngram #: 3313
image bzip2 ngram #: 5950
image mysql ngram #: 21211
image mysql ngram #: 39267
image mysql ngram #: 47973
image mysql ngram #: 57454
image gcc ngram #: 160986
image gcc ngram #: 182640
image httpd ngram #: 3238
image httpd ngram #: 5328
image httpd ngram #: 8002
image lowriter ngram #: 2083829
image lowriter ngram #: 2834289
image lowriter ngram #: 4085724
image node ngram #: 22280
image node ngram #: 23125
image nginx ngram #: 169
image nginx ngram #: 424
image nginx ngram #: 532
image ghostscript ngram #: 4684
image ghostscript ngram #: 113998
image ghostscript ngram #: 266548
image ghostscript ngram #: 280294
image ghostscript ngram #: 297234
image redis ngram #: 4662
image redis ngram #: 7509
image redis ngram #: 7509
image redis ngram #: 7509
image mongodb ngram #: 238897
image mongodb ngram #: 472457
image mongodb ngram #: 619903
image mongodb ngram #: 856087
image gzip 

# 중복제거?

In [411]:
# Ngram 총 개수
b_ngramList_total = set()
for imgName in imgList:
    b_ngramList_total = b_ngramList_total | b_ngramList[imgName]
print(f"Total benign ngram #: {len(b_ngramList_total)}")

Total benign ngram #: 17995008


## Token 크기 별 Image N-gram 

In [329]:
# df Ngram_num 구하는 함수
def ngram_num(df):
    for index, row in df.iterrows():
        df["Ngram_num"][index] = len(row.Ngram)
    return df

In [330]:
def make_save_ngram(ngramtmp, imgName):
    import pandas as pd
    tmp_ngram = pd.DataFrame(columns=["N","Ngram","Ngram_num","fp", "fp_num"])
    # df 만들기
    for N, ngramList in ngramtmp.items():
        tmpdict={"N":0, "Ngram":[], "Ngram_num": 0, "fp":[],"fp_num":0}
        if not ngramList:
            break
        tmpdict["N"]=N
        tmpdict["Ngram"] = ngramList
        tmp_ngram = tmp_ngram.append(tmpdict, ignore_index=True)
        # df Ngram_num 구하기
        tmp_ngram= ngram_num(tmp_ngram)
        # excel 저장
        tmp_ngram.to_excel(f"ngram_num_per_ngram_token_size({imgName}).xlsx")

In [331]:
for imgName in imgList:
    print(imgName)
    ngramtmp = dict()
    for i in range(2,66):
        ngramtmp[i]=list()
    # image 당 n-gram 토큰 크기별로 모으기
    for ngram in b_ngramList[imgName]:
        N = len(ngram.split(","))
        ngramtmp[N].append(ngram)
    # df 만들고 저장
    make_save_ngram(ngramtmp, imgName)

tomcat
bzip2
mysql
gcc
httpd
node
nginx
redis
mongodb
gzip
mariadb
openjdk


# Exploit Code N-gram

In [396]:
len(m_ngram["Ngram_syscall"][0])

61

In [394]:
import pickle

## Load pickle
# with open("/opt/output/perm/analysis/soft_dedup_ngramdf.pkl","rb") as f:
with open("soft_dedup_ngramdf.pkl","rb") as f:
    m_ngram = pickle.load(f)
m_ngram

,N,Ngram_syscall,num_of_exploit,EIDs
0,61,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
1,49,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"
2,45,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
3,43,"[9, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218, 2...",2,"[40616, 43127]"
4,43,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
...,...,...,...,...
473,2,"[257, 0]",16,"[15024, 15774, 3, 35370, 40616, 40871, 42275, ..."
474,2,"[8, 0]",8,"[12, 20979, 27297, 3, 43127, 45553, 50808, 9083]"
475,2,"[3, 59]",9,"[39230, 40871, 43418, 47170, 6851, 9479, 9542]"
476,2,"[8, 5]",4,"[12, 3, 43127, 50541]"


# Exploit(106)과 Image Set 분석

## Exploit(106) Syscall Set

In [109]:
# 모든 익스플로잇 path laod
import subprocess
import json

pathFile = subprocess.check_output(f"find ../data/path/*.json", shell=True).decode().strip().split()
EIDList = list(map(lambda x: x.replace("../data/path/","").replace(".json",""), pathFile))
pathList = dict() # 익스플로잇 코드 path dictionary의 dictionary - pathList[EID][path] = path 파일 정보
for EID in EIDList:
    with open(f"../data/path/{EID}.json", "r") as f:
        pathList[EID] = dict()
        pathList[EID]["path"] = json.load(f)

In [111]:
# dict {EID: all path list} 만들어 주기
eid_all_path_dict = dict()
for EID, pathset in pathList.items():
    eid_all_path_dict[EID] = list()
    for userfunc, pathset in pathset['path'].items():
        if not len(pathset) == 0:
            eid_all_path_dict[EID].extend(pathset)

In [121]:
# 중복 제거
eid_all_path_set = dict()
tmpSet = set()
for EID, pathset in eid_all_path_dict.items():
    eid_all_path_set[EID] = list()
    for path in pathset:
        tmpSet.add(",".join(path))
    for path in tmpSet:
        eid_all_path_set[EID].append(path.split(","))
    tmpSet=set()

In [125]:
m_sysSet = set()
for EID, pathset in eid_all_path_set.items():
    for path in pathset:
        m_sysSet = m_sysSet | set(path)
len(m_sysSet)

142

In [126]:
def false_positive_set(m, b):
    falsePositive_sysSet = m & b
    print(f"false positive syscall #: {len(falsePositive_sysSet)}")
    tmp = set()
    for syscall in falsePositive_sysSet:
        tmp.add(sysnum_convert[syscall])
    print(tmp)

In [412]:
# all
print("="*10,"<all image>","="*10)
false_positive_set(m_sysSet, b_sysSet_total)
for imgName in imgList:
    print("="*10,f"<{imgName}>","="*10)
    false_positive_set(m_sysSet, b_sysSet[imgName])

========== <all image> ==========
false positive syscall #: 80
{'write', 'prctl', 'epoll_ctl', 'unlink', 'sendmmsg', 'newfstatat', 'arch_prctl', 'newfstat', 'rt_sigprocmask', 'mkdir', 'link', 'getgid', 'getsockname', 'execve', 'exit_group', 'recvmsg', 'clock_nanosleep', 'exit', 'fcntl', 'getpid', 'sendmsg', 'prlimit64', 'getdents64', 'readlink', 'mmap', 'pipe', 'rt_sigreturn', 'sendfile64', 'accept', 'poll', 'openat', 'set_robust_list', 'newstat', 'statfs', 'lseek', 'epoll_wait', 'wait4', 'getsockopt', 'rt_sigaction', 'ioctl', 'sendto', 'chown', 'nanosleep', 'socketpair', 'umask', 'brk', 'getppid', 'bind', 'shutdown', 'newuname', 'connect', 'timerfd_settime', 'sched_yield', 'recvfrom', 'mprotect', 'dup', 'madvise', 'getegid', 'rmdir', 'clone', 'futex', 'getcwd', 'access', 'setsockopt', 'geteuid', 'setrlimit', 'close', 'rename', 'chmod', 'socket', 'read', 'sigaltstack', 'getuid', 'munmap', 'ftruncate', 'listen', 'open', 'pread64', 'gettid', 'set_tid_address'}
========== <tomcat> =======

# Exploit(58)과 Image Set 분석

## Exploit(58) Syscall Set 

In [93]:
m_sysNameSet = {'shmat', 'statfs', 'pipe', 'mprotect', 'setgroups', 'prctl', 'gettid', 'wait4', 'inotify_add_watch', 'getpid', 'modify_ldt', 'listen', 'setreuid', 'setresgid', 'newstat', 'access', 'chmod', 'setsid', 'clock_nanosleep', 'connect', 'sigaltstack', 'rt_sigprocmask', 'mmap', 'setsockopt', 'sendmmsg', 'sendmsg', 'alarm', 'time', 'mlock', 'getegid', 'sched_setaffinity', 'inotify_init1', 'bind', 'capget', 'setrlimit', 'pread64', 'msgget', 'set_robust_list', 'dup', 'sched_setscheduler', 'futex', 'getpriority', 'getuid', 'dup2', 'sched_yield', 'kill', 'chdir', 'msgctl', 'setresuid', 'read', 'timerfd_settime', 'mkdir', 'getcwd', 'recvmsg', 'accept', 'epoll_ctl', 'setpriority', 'set_tid_address', 'fcntl', 'openat', 'open', 'lseek', 'getgid', 'epoll_wait', 'chown', 'shmget', 'syslog', 'sendto', 'prlimit64', 'poll', 'newuname', 'getsockname', 'setgid', 'readlink', 'rt_sigaction', 'newfstat', 'rt_sigreturn', 'flock', 'ioctl', 'getsockopt', 'splice', 'unlink', 'close', 'socketcall', 'execve', 'socket', 'munmap', 'mq_notify', 'madvise', 'clone', 'ftruncate', 'futimesat', 'brk', 'fchmod', 'waitid', 'write', 'epoll_create', 'sendfile64', 'getppid', 'vmsplice', 'rename', 'exit_group', 'geteuid', 'capset', 'timerfd_create', 'socketpair', 'arch_prctl', 'setuid'}
m_sysSet = set()
for syscall in m_sysNameSet:
    m_sysSet.add(sysname_convert[syscall])
len(m_sysSet)

108

In [335]:
# all
print("="*10,"<all image>","="*10)
false_positive_set(m_sysSet, b_sysSet_total)
for imgName in imgList:
    print("="*10,f"<{imgName}>","="*10)
    false_positive_set(m_sysSet, b_sysSet[imgName])

========== <all image> ==========
false positive syscall #: 73
{'wait4', 'sendmsg', 'lseek', 'getdents64', 'getsockopt', 'madvise', 'getegid', 'prlimit64', 'epoll_wait', 'clone', 'write', 'futex', 'readlink', 'rt_sigaction', 'ioctl', 'prctl', 'epoll_ctl', 'getcwd', 'access', 'sendto', 'unlink', 'mprotect', 'setsockopt', 'sendmmsg', 'newfstatat', 'nanosleep', 'arch_prctl', 'socketpair', 'umask', 'mmap', 'newfstat', 'brk', 'shutdown', 'geteuid', 'newuname', 'setrlimit', 'connect', 'rt_sigprocmask', 'close', 'rename', 'mkdir', 'chmod', 'rt_sigreturn', 'timerfd_settime', 'sched_yield', 'getgid', 'socket', 'sendfile64', 'recvfrom', 'accept', 'read', 'getsockname', 'execve', 'sigaltstack', 'exit_group', 'poll', 'recvmsg', 'clock_nanosleep', 'getuid', 'exit', 'dup', 'munmap', 'ftruncate', 'openat', 'open', 'pread64', 'fcntl', 'set_robust_list', 'getpid', 'set_tid_address', 'gettid', 'newstat', 'statfs'}
========== <tomcat> ==========
false positive syscall #: 20
{'epoll_wait', 'getsockopt', '

# Token 크기 별 N-gram 개수 그래프 (benign)

In [413]:
# benign 
import pandas as pd
b_ngram = pd.DataFrame(columns=["N","Ngram","num_of_ngram","fp"])

tmpdict = dict()
for ngram in b_ngramList_total:
    N = len(ngram.split(","))
    if not N in tmpdict.keys():
        tmpdict[N] = {"Ngram":[], "num_of_ngram":1, "fp": 0}
        tmpdict[N]["Ngram"].append(ngram)
    else:
        tmpdict[N]["Ngram"].append(ngram)
    
tmpdict_for_df={"N":"", "Ngram":[], "num_of_ngram":1, "fp": 0}
for N, tmp in tmpdict.items():
    tmpdict_for_df["N"] = N
    tmpdict_for_df["Ngram"] = tmp["Ngram"]
    tmpdict_for_df["num_of_ngram"] = len(tmp["Ngram"])
    b_ngram = b_ngram.append(tmpdict_for_df, ignore_index=True)
    tmpdict_for_df={"N":"", "Ngram":[], "num_of_ngram":1, "fp": 0}
    
b_ngram = b_ngram.sort_values(by=["N"], ascending=True)
b_ngram

,N,Ngram,num_of_ngram,fp
63,2,"[87,6, 53,72, 3,16, 158,61, 102,21, 39,204, 49...",1376,0
61,3,"[96,96,12, 228,228,21, 202,10,56, 14,1,1, 202,...",4514,0
62,4,"[0,3,228,1, 72,56,56,56, 9,4,9,257, 13,39,39,2...",11050,0
60,5,"[228,14,14,10,14, 228,228,47,233,228, 14,14,20...",20849,0
56,6,"[99,10,99,96,10,96, 228,186,228,186,17,228, 22...",34081,0
...,...,...,...,...
11,61,"[1,1,0,0,1,1,1,1,0,0,1,0,1,0,1,1,1,0,1,0,1,1,1...",389540,0
6,62,"[202,202,228,186,202,202,228,228,228,228,228,2...",391011,0
35,63,"[228,228,228,10,10,228,228,228,228,228,228,228...",392461,0
31,64,"[202,96,228,228,228,228,202,202,202,202,202,20...",393872,0


In [414]:
for index, row in m_ngram.iterrows():
    m_ngram_str = ",".join(row.Ngram_syscall)
    ret = b_ngram["Ngram"][b_ngram["N"] == row.N].to_list()[0]
    if m_ngram_str in ret:
        if row.N == 11:
            print(m_ngram_str)
        b_ngram["fp"][b_ngram["N"] == row.N] = b_ngram["fp"][b_ngram["N"] == row.N]+1
b_ngram = b_ngram.sort_values(by=["N"], ascending=True)
b_ngram.to_excel("ngram_num_per_ngram_token_size(benign).xlsx")
b_ngram

257,5,9,3,257,0,5,9,9,9,9


,N,Ngram,num_of_ngram,fp
63,2,"[87,6, 53,72, 3,16, 158,61, 102,21, 39,204, 49...",1376,77
61,3,"[96,96,12, 228,228,21, 202,10,56, 14,1,1, 202,...",4514,50
62,4,"[0,3,228,1, 72,56,56,56, 9,4,9,257, 13,39,39,2...",11050,10
60,5,"[228,14,14,10,14, 228,228,47,233,228, 14,14,20...",20849,4
56,6,"[99,10,99,96,10,96, 228,186,228,186,17,228, 22...",34081,2
...,...,...,...,...
11,61,"[1,1,0,0,1,1,1,1,0,0,1,0,1,0,1,1,1,0,1,0,1,1,1...",389540,0
6,62,"[202,202,228,186,202,202,228,228,228,228,228,2...",391011,0
35,63,"[228,228,228,10,10,228,228,228,228,228,228,228...",392461,0
31,64,"[202,96,228,228,228,228,202,202,202,202,202,20...",393872,0


In [252]:
# 11-gram 어떤 시퀀스인지 조사
tmpstr = "257,5,9,3,257,0,5,9,9,9,9"
tmptmp = tmpstr.split(",")
ttt = []
for syscall in tmptmp:
    ttt.append(sysnum_convert[syscall])
print(ttt)

['openat', 'newfstat', 'mmap', 'close', 'openat', 'read', 'newfstat', 'mmap', 'mmap', 'mmap', 'mmap']


# Exploit Coverage / Appilcation Operation Coverage

In [339]:
import pandas as pd
m_ratio = pd.DataFrame(columns=["N","Ngram","Ngram_num","Exploit","Exploit_num","Ratio"])
b_ratio = pd.DataFrame(columns=["N","Ngram","Ngram_num","Op","Op_num","Ratio"])

In [345]:
# exploit path 정리 (list에서 str으로 변환)
m_path=dict()
for eid, pathset in eid_all_path_set.items():
    tmppath = list()
    for path in  pathset:
        tmppath.append(",".join(path))
    m_path[eid]=tmppath

In [347]:
# Benign path 정리 (list에서 str으로 변환)
b_path=dict()
ops = seqNumList.keys()
for op, seq in seqNumList.items():
    b_path[op]=",".join(seq)

In [356]:
m_ngram

,N,Ngram_syscall,num_of_exploit,EIDs
477,2,"[12, 41]",3,"[3, 50541, 926]"
470,2,"[33, 59]",2,"[25450, 35161]"
468,2,"[104, 102]",2,"[14814, 926]"
466,2,"[8, 1]",5,"[27297, 37167, 40616, 40871, 47170]"
396,2,"[0, 5]",35,"[12, 1397, 14814, 15023, 15024, 15285, 15704, ..."
...,...,...,...,...
4,43,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
3,43,"[9, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218, 2...",2,"[40616, 43127]"
2,45,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
1,49,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"


In [361]:
# 오름차순으로 정렬
m_ngram = m_ngram.sort_values(by=["N"], ascending=True)
block_ngram = dict()
block_EIDs = dict()
for N in set(m_ngram.N.to_list()):
    block_ngram[N]=set()
    block_EIDs[N]=set()
for index, row in m_ngram.iterrows():
    block_ngram[row.N].add(",".join(row.Ngram_syscall))
    block_EIDs[row.N] = block_EIDs[row.N] | set(row.EIDs)

In [390]:
# M(exploit) coverage 조사
m_ratio = pd.DataFrame(columns=["N","Ngram","Ngram_num","Exploit","Exploit_num","Ratio"])
TOTAL=106
tmpratio = {"N":0,"Ngram":[],"Ngram_num":0,"Exploit":[],"Exploit_num":0,"Ratio":0}
for N in set(m_ngram.N.to_list()):
    tmpratio["N"] = N
    tmpratio["Ngram"] = block_ngram[N]
    tmpratio["Ngram_num"] = len(block_ngram[N])
    tmpratio["Exploit"] = block_EIDs[N]
    if N == 2: 
        tmpratio["Exploit_num"] = 106
        tmpratio["Ratio"] = 106/TOTAL*100
    else:
        tmpratio["Exploit_num"] = len(block_EIDs[N])
        tmpratio["Ratio"] = len(block_EIDs[N])/TOTAL*100
    # pd에 append
    m_ratio = m_ratio.append(tmpratio, ignore_index=True)
# 엑셀에 저장
m_ratio.to_excel(f"exploit_coverage.xlsx")
m_ratio

,N,Ngram,Ngram_num,Exploit,Exploit_num,Ratio
0,2,"{230,3, 3,56, 257,1, 272,272, 46,3, 9,11, 9,25...",163,"{50541, 33336, 25450, 37167, 22362, 4460, 2006...",106,100.000000
1,3,"{56,59,231, 12,12,3, 257,1,91, 230,0,3, 49,50,...",108,"{50541, 33336, 25450, 37167, 22362, 4460, 2006...",92,86.792453
2,4,"{118,9,257,59, 321,321,53,54, 101,101,101,61, ...",52,"{50541, 33336, 37167, 22362, 4460, 2006, 37292...",78,73.584906
3,5,"{5,5,8,0,8, 3,12,12,257,5, 42,44,54,44,3, 257,...",18,"{50541, 6851, 33336, 42275, 43127, 1397, 45516...",49,46.226415
4,6,"{257,4,257,4,13,13, 12,12,257,0,5,3, 21,21,41,...",20,"{50541, 33336, 20000, 42275, 45516, 17787, 207...",54,50.943396
5,7,"{9,9,9,9,3,10,11, 54,54,54,54,54,54,54, 56,62,...",9,"{50541, 33336, 17787, 20721, 22362, 2006, 4004...",34,32.075472
6,8,"{41,12,12,9,54,54,54,3, 12,12,12,12,160,22,3,1...",8,"{50541, 33336, 42275, 43127, 17787, 20721, 159...",34,32.075472
7,9,"{12,12,257,5,0,5,0,5,0, 257,1,3,257,1,3,257,1,...",6,"{33336, 17787, 2006, 40049, 44303, 38390, 3923...",27,25.471698
8,10,"{12,12,5,0,5,0,5,0,5,0, 257,89,1,41,257,4,8,1,...",6,"{50541, 33336, 42275, 17787, 20721, 15944, 400...",27,25.471698
9,11,"{257,5,9,3,257,0,5,9,9,9,9, 41,54,54,41,21,21,...",5,"{15285, 50541, 14814, 33336, 33516, 15704, 150...",25,23.584906


In [397]:
# B(image) coverage 조사
b_ratio = pd.DataFrame(columns=["N","Ngram","Ngram_num","Op","Op_num","Ratio"])
TOTAL=len(seqNumList.keys()) # 36
for N in set(m_ngram.N.to_list()):
    tmpratio = {"N":0,"Ngram":[],"Ngram_num":0,"Op":set(),"Op_num":0,"Ratio":0}
    tmpratio["N"] = N
    tmpratio["Ngram"] = block_ngram[N]
    tmpratio["Ngram_num"] = len(block_ngram[N])
    for ngram in block_ngram[N]:
        for op in list(b_path.keys()):
            if ngram in b_path[op]:
                tmpratio["Op"].add(op)
    tmpratio["Op_num"] = len(tmpratio["Op"])
    tmpratio["Ratio"] = tmpratio["Op_num"]/TOTAL*100
    # pd에 append
    b_ratio = b_ratio.append(tmpratio, ignore_index=True)
# 엑셀에 저장
b_ratio.to_excel(f"benign_coverage.xlsx")
b_ratio

,N,Ngram,Ngram_num,Op,Op_num,Ratio
0,2,"{230,3, 3,56, 257,1, 272,272, 46,3, 9,11, 9,25...",163,"{bzip2_unzip, mysql_u, mysql_d, mysql_r, gzip_...",36,100.000000
1,3,"{56,59,231, 12,12,3, 257,1,91, 230,0,3, 49,50,...",108,"{bzip2_unzip, mysql_u, gzip_zip, mysql_d, mysq...",30,83.333333
2,4,"{118,9,257,59, 321,321,53,54, 101,101,101,61, ...",52,"{bzip2_unzip, openjdk_strace2, mysql_u, mysql_...",16,44.444444
3,5,"{5,5,8,0,8, 3,12,12,257,5, 42,44,54,44,3, 257,...",18,"{openjdk_strace2, mysql_u, mongodb_c, mysql_d,...",11,30.555556
4,6,"{257,4,257,4,13,13, 12,12,257,0,5,3, 21,21,41,...",20,"{openjdk_strace2, gzip_zip, gzip_unzip, gcc_st...",6,16.666667
5,7,"{9,9,9,9,3,10,11, 54,54,54,54,54,54,54, 56,62,...",9,{},0,0.000000
6,8,"{41,12,12,9,54,54,54,3, 12,12,12,12,160,22,3,1...",8,{},0,0.000000
7,9,"{12,12,257,5,0,5,0,5,0, 257,1,3,257,1,3,257,1,...",6,{},0,0.000000
8,10,"{12,12,5,0,5,0,5,0,5,0, 257,89,1,41,257,4,8,1,...",6,{},0,0.000000
9,11,"{257,5,9,3,257,0,5,9,9,9,9, 41,54,54,41,21,21,...",5,{mysql_c},1,2.777778


In [386]:
# 각 Operation 별 FP 비율 조사


true
